In [ ]:
import hax
from hax import cuts
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from matplotlib.colors import LogNorm
import time
import datetime

In [ ]:
def get_run_list(path, start_at=None, stop_at=None):
    '''
    Description pending
    '''
    file_list = np.sort(os.listdir(path))
    
    file_list = [file[:-5] for file in file_list if file[-5:]=='.root']
    
    if start_at:
        start_index = np.where(np.array(file_list) == start_at)[0][0]
        file_list = file_list[start_index:]
    if stop_at:
        stop_index = np.where(np.array(file_list) == stop_at)[0][0]
        file_list = file_list[:stop_index+1]
    print('Run list contains %d files' % len(file_list))
    return file_list

In [ ]:
def get_time(d, run_names):    
    next_run_indices = np.where(np.diff(d['event_number'].values) < 0)[0]
    def num(x):
        return len([a for a in next_run_indices if a<x])
    run_numbers = [num(r) for r in d.index]
    d['run_number'] = run_numbers

    all_timestamps = []
    for run_number, run_name in zip(np.unique(run_numbers), run_names):
        d_sub = d[d['run_number']==run_number]
        all_timestamps.append(
            ((d_sub['event_time'].values - d_sub['event_time'].iloc[0]) * 4) * 1e-9 +
            time.mktime(datetime.datetime.strptime(run_name, "%y%m%d_%H%M%S").timetuple()))

    d['unix_time'] = np.concatenate(all_timestamps)
    d['time_since_start'] = d['unix_time'] - d['unix_time'].iloc[0]
    return d

In [ ]:
class XAMSProperties(hax.minitrees.TreeMaker):
    __version__ = '0.1.0'
    
    extra_branches = ['peaks.*', 'peaks.range_area_decile*', 
                      'interactions*']
    
    def extract_data(self, event):
        
        peak_types = ['s1', 's2']
        peak_properties = ['area_fraction_top', 'area', 'center_time', 'n_saturated_channels']
        extra_properties = ['range_50p_area', 'range_70p_area']
        all_properties = peak_properties + extra_properties
        result_vars = [_pt + '_' + _pp for _pt in peak_types for _pp in all_properties ]
        result = {k: float('nan') for k in result_vars}
        
        interaction_properties = ['drift_time']
        for _prop in interaction_properties:
            result[_prop] = float('nan')
            
        s1s = []
        s2s = []
        
        for peak in event.peaks:
            if peak.type == 's1':
                s1s.append(peak)
            if peak.type == 's2':
                s2s.append(peak)
        
        s1s = sorted(s1s, key = lambda peak: - peak.area)
        s2s = sorted(s2s, key = lambda peak: - peak.area)

        
        # Now load properies
        result['n_s1s'] = len(event.s1s)
        result['n_s2s'] = len(event.s2s)
        
        
        for peak_name, peak_list in zip(['s1', 's2'], [s1s, s2s]):
            if len(peak_list) == 0:
                continue
            # First peak in list sorted by area
            peak = peak_list[0]
            for _prop in peak_properties:
                result[peak_name + '_' + _prop] = getattr(peak, _prop)
            for i in range(10):
                if ('range_%d0p_area' % i) in extra_properties:
                    result[peak_name + '_' + 'range_%d0p_area' % i] = list(peak.range_area_decile)[i]
        
        if len(s2s) > 1:
            result['largest_other_s2'] = getattr(s2s[1], 'area')
        else:
            result['largest_other_s2'] = 0.
            
        result['drift_time'] = result['s2_center_time'] - result['s1_center_time']
        



#             for peak_type, peak_index in zip(peak_types, [s1,s2]):
#                 for _prop in peak_properties:
#                     result[peak_name + '_' + _prop] = getattr(event.peaks[peak_index], _prop)

#                 if 'range_50p_area' in extra_properties:
#                     result[peak_name + '_' + 'range_50p_area'] = list(event.peaks[peak_pos].range_area_decile)[5]
        
        return result